In [29]:
import openai
from qdrant_client import QdrantClient

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

### Download an example reference data point from LangSmith

In [30]:
client = Client()

In [31]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [32]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('83a7dc72-f71e-4a0a-84b6-2bccd73085cb'), created_at=datetime.datetime(2026, 1, 20, 19, 37, 40, 788414, tzinfo=TzInfo(0)), modified_at=datetime.datetime(2026, 1, 20, 19, 37, 40, 788414, tzinfo=TzInfo(0)), example_count=38, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'Windows-10-10.0.19045-SP0', 'sdk_version': '0.6.4', 'runtime_version': '3.14.2', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': None}})

In [33]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': "Yes. The Tenda A33 AX3000 WiFi Extender (B0BZ5R7CVP) notes that while it's compatible with most routers, it may not reliably extend a mesh network and recommends adding a mesh node instead.",
 'reference_context_ids': ['B0BZ5R7CVP'],
 'reference_descriptions': ['Tenda A33 AX3000 WiFi 6 Extender, WiFi Booster WiFi Range Extender, 2.4/5GHz Dual Band WiFi Extender with Ethernet Port, AP Mode, WPS Easy Setup, WiFi Extenders Signal Booster for Home Improved WiFi Coverage - With 2 * 5dbi dual-band antennas, the Tenda A33 wifi extender can effectively boost your wifi signal for up to 2100 sq. ft coverage , providing you a stable wifi connection at your home with no dead zone. Fast Speed - Tenda A33 WiFi 6 Range Extender provides maximum speeds of 574Mbps on the 2.4GHz band and 2402Mbps on the 5GHz band, delivering you an unrivaled network experience at every corner of your house. More Devices, Less Latency - Compared with the previous WiFi 5 generation, WiFi 6 technology ena

In [34]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Is there any product in the list that explicitly states it cannot extend a mesh network reliably?'}

In [35]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

### RAG Pipeline

In [36]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content


def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [37]:
rag_pipeline("Can I get some charger?", top_k=5)

{'answer': 'Yes. Here are charger cable options currently available:\n\n- B0BYYLJRHT: iPhone Charger Cord Lightning Cables, 3-pack, 3ft, Apple MFi certified. Durable with reinforced joints. Compatible with iPhone models and iPad/iPod. Includes 3 cables.\n- B0BFPZGYLD: 5 in 1 USB C to Multi Charging Cable, 10ft (3m/10ft), USB A to Lightning, USB C to Lightning, USB C, Micro USB. Not for iPad. Multi-device charging; 6 cables in total inside. MFi certified.\n- B09TNXY54Y: MUXA 6 Pack, 3/3/6/6/10/10 ft colorful Nylon Lightning cables, MFi certified. Wide iPhone compatibility; various lengths.\n- B0BV6PWVCG: GREPHONE 2 Pack USB C to Lightning Cable, 6 ft, MFi certified, supports fast charging. Includes two 6ft cables.\n- B0BGDQLZD2: Mixblu Charger Cable Replacement for Fitbit Inspire 3 (2 pack, 3.3ft).\n\nIf you tell me how many you’d like and any preferred length or device (iPhone only vs multi-use), I can suggest the best match and link you to a specific option.',
 'question': 'Can I get 

### RAGAS metrics

In [38]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

C:\Users\Misa\AppData\Local\Temp\ipykernel_257984\1973645927.py:2: DeprecationWarning: Importing IDBasedContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextPrecision
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
C:\Users\Misa\AppData\Local\Temp\ipykernel_257984\1973645927.py:2: DeprecationWarning: Importing IDBasedContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import IDBasedContextRecall
  from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy
C:\Users\Misa\AppData\Local\Temp\ipykernel_257984\1973645927.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use

In [39]:
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

C:\Users\Misa\AppData\Local\Temp\ipykernel_257984\2270190839.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
C:\Users\Misa\AppData\Local\Temp\ipykernel_257984\2270190839.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [40]:
reference_input

{'question': 'Is there any product in the list that explicitly states it cannot extend a mesh network reliably?'}

In [41]:
reference_output

{'ground_truth': "Yes. The Tenda A33 AX3000 WiFi Extender (B0BZ5R7CVP) notes that while it's compatible with most routers, it may not reliably extend a mesh network and recommends adding a mesh node instead.",
 'reference_context_ids': ['B0BZ5R7CVP'],
 'reference_descriptions': ['Tenda A33 AX3000 WiFi 6 Extender, WiFi Booster WiFi Range Extender, 2.4/5GHz Dual Band WiFi Extender with Ethernet Port, AP Mode, WPS Easy Setup, WiFi Extenders Signal Booster for Home Improved WiFi Coverage - With 2 * 5dbi dual-band antennas, the Tenda A33 wifi extender can effectively boost your wifi signal for up to 2100 sq. ft coverage , providing you a stable wifi connection at your home with no dead zone. Fast Speed - Tenda A33 WiFi 6 Range Extender provides maximum speeds of 574Mbps on the 2.4GHz band and 2402Mbps on the 5GHz band, delivering you an unrivaled network experience at every corner of your house. More Devices, Less Latency - Compared with the previous WiFi 5 generation, WiFi 6 technology ena

In [42]:
result = rag_pipeline(reference_input["question"])

In [43]:
result

{'answer': 'Yes. The Tenda A33 WiFi 6 Range Extender notes that if you want to extend a mesh network, adding a mesh node is recommended because most mesh systems remove the "AP Client" feature that is necessary for a WiFi extender to work. This implies it cannot reliably extend a mesh network in the standard way.',
 'question': 'Is there any product in the list that explicitly states it cannot extend a mesh network reliably?',
 'retrieved_context_ids': ['B0BZ5R7CVP',
  'B0BR8Q2PXN',
  'B0BP9Z159S',
  'B0BFPZGYLD',
  'B09TNXY54Y'],
 'retrieved_context': ['Tenda A33 AX3000 WiFi 6 Extender, WiFi Booster WiFi Range Extender, 2.4/5GHz Dual Band WiFi Extender with Ethernet Port, AP Mode, WPS Easy Setup, WiFi Extenders Signal Booster for Home Improved WiFi Coverage - With 2 * 5dbi dual-band antennas, the Tenda A33 wifi extender can effectively boost your wifi signal for up to 2100 sq. ft coverage , providing you a stable wifi connection at your home with no dead zone. Fast Speed - Tenda A33 W

In [44]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [45]:
await ragas_faithfulness(result, "")

1.0

In [46]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [47]:
await ragas_responce_relevancy(result, "")

np.float64(0.5832396577951091)

In [48]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [49]:
await ragas_context_precision_id_based(result, reference_output)

0.2

In [50]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [51]:
await ragas_context_recall_id_based(result, reference_output)

1.0